In [ ]:
# stdlib
import os

# third party
import numpy as np
import requests

# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["SERVER_PORT"] = "8081"

In [ ]:
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
server_port = os.environ.get("SERVER_PORT", "auto")
print(environment, server_port)

# Verify Version

In [ ]:
PROJECT_NAME = "syft"
PYPI_JSON_URL = f"https://pypi.org/pypi/{PROJECT_NAME}/json"


def get_latest_pypi_version():
    response = requests.get(PYPI_JSON_URL)
    data = response.json()
    return data["info"]["version"]


latest_deployed_version = get_latest_pypi_version()

In [ ]:
# this notebook should only be used to run the latest deployed version of syft
# the notebooks after this (1a/1b and 2), will test migrating from that latest version
print(
    f"latest deployed version: {latest_deployed_version}, installed version: {sy.__version__}"
)
# assert (
#     latest_deployed_version == sy.__version__
# ), f"{latest_deployed_version} does not match installed version {sy.__version__}"

# Launch Server

In [ ]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=True,
    n_consumers=2,
    create_producer=True,
    port=server_port,
)

In [ ]:
client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
client.register(
    email="ds@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

# Prepare some data to be migrated

## External registry

In [ ]:
external_registry = test_settings.get("external_registry", default="docker.io")
external_registry

In [ ]:
result = client.api.services.image_registry.add(external_registry)
result

In [ ]:
image_registry_list = client.api.services.image_registry.get_all()
image_registry_list

In [ ]:
local_registry = image_registry_list[0]
local_registry

## custom workerpool

In [ ]:
# Get base worker image
base_worker_image = client.images.get_all()[0]
base_worker_image

In [ ]:
worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery
""".strip()

docker_config = sy.DockerWorkerConfig(dockerfile=worker_dockerfile)

print(docker_config)

In [ ]:
assert docker_config.dockerfile == worker_dockerfile

In [ ]:
submit_result = client.api.services.worker_image.submit(worker_config=docker_config)

assert isinstance(submit_result, sy.SyftSuccess)
custom_image = submit_result.value

In [ ]:
docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)
docker_tag

In [ ]:
build_result = client.api.services.worker_image.build(
    image_uid=custom_image.id,
    tag=docker_tag,
    registry_uid=local_registry.id,
)

assert isinstance(build_result, sy.SyftSuccess)
build_result

In [ ]:
push_result = client.api.services.worker_image.push(custom_image.id)
push_result

In [ ]:
custom_pool_name = "custom-bigquery"

worker_pool_res = client.api.services.worker_pool.launch(
    pool_name=custom_pool_name,
    image_uid=custom_image.id,
    num_workers=4,
)

In [ ]:
worker_pool_res

In [ ]:
images = client.services.worker_image.get_all()

assert len(images) == 2

In [ ]:
worker_pools = client.worker_pools.get_all()

assert len(worker_pools) == 2

worker_pools

## Prebuilt custom workerpool

In [ ]:
docker_tag

In [ ]:
docker_config = sy.PrebuiltWorkerConfig(tag=docker_tag)
docker_config

In [ ]:
result = client.api.services.worker_image.submit(worker_config=docker_config)
assert isinstance(result, sy.SyftSuccess)

prebuilt_image = result.value
prebuilt_image

In [ ]:
prebuilt_pool_name = "bigquery-prebuilt"
result = client.api.services.worker_pool.launch(
    pool_name=prebuilt_pool_name,
    image_uid=prebuilt_image.id,
    num_workers=1,
)
result

In [ ]:
client.api.services.worker_image.get_all()

In [ ]:
images = client.api.services.worker_image.get_all()

assert len(images) == 2

In [ ]:
worker_pools = client.worker_pools.get_all()

assert len(worker_pools) == 3

## Dataset, Code, Job

In [ ]:
client_ds = server.login(email="ds@openmined.org", password="pw")

In [ ]:
dataset = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=np.array([10, 11, 12, 13, 14]),
            data=np.array([[15, 16, 17, 18, 19] for _ in range(100_000)]),
            mock_is_real=True,
        )
    ],
)

client.upload_dataset(dataset)

In [ ]:
data_low = client_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low, worker_pool_name="bigquery-prebuilt")
def compute_mean(datasite, data) -> float:
    # launch job on prebuilt workerpool
    print("Computing mean...")
    return data

In [ ]:
req = client_ds.code.request_code_execution(compute_mean)

In [ ]:
client.requests[0].approve()

In [ ]:
job = client_ds.code.compute_mean(data=data_low, blocking=False)

In [ ]:
res = job.wait()

In [ ]:
assert isinstance(res.get(), np.ndarray)

In [ ]:
# todo: add more data

In [ ]:
if server.server_type.value == "python":
    server.land()